In [33]:
import cv2
import numpy as np
import pandas as pd
import os

In [39]:
recording_dir = 'recordings/'
camera_names = [f'Camera{i}_007.mp4' for i in range(7)]

df = pd.DataFrame()

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)

i=0

for name in camera_names:
    video_name = os.path.join(recording_dir, name)
    cap = cv2.VideoCapture(video_name)
    print(f'Processing {video_name}')
    median_x = []
    median_y = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        mask = cv2.inRange(frame, lower_red, upper_red)
        detected_ball = cv2.bitwise_and(frame, frame, mask=mask)    
        red_coordinates = np.column_stack(np.where((frame[:, :, 0] >= lower_red[0]) & (frame[:, :, 0] <= upper_red[0]) &
        (frame[:, :, 1] >= lower_red[1]) & (frame[:, :, 1] <= upper_red[1]) &
        (frame[:, :, 2] >= lower_red[2]) & (frame[:, :, 2] <= upper_red[2])
        ))    
        red_coordinates[:, 0] = frame.shape[0] - red_coordinates[:, 0]
        if red_coordinates.size == 0:
            median_x.append(np.nan)
            median_y.append(np.nan)
        else:
            median_x.append(np.median(red_coordinates[:, 1]))
            median_y.append(np.median(red_coordinates[:, 0]))
    
    
    df[f'u{i}'] = median_x
    df[f'v{i}'] = median_y
    cap.release()
    
    
    
    i+=1
# Make sure the 'data' directory exists
if not os.path.exists('data'):
    os.makedirs('data')

# Save the dataframe to a CSV file
df.to_csv('data/red_ball_coordinates.csv', index=False)

Processing recordings/Camera0_007.mp4
Processing recordings/Camera1_007.mp4
Processing recordings/Camera2_007.mp4
Processing recordings/Camera3_007.mp4
Processing recordings/Camera4_007.mp4
Processing recordings/Camera5_007.mp4
Processing recordings/Camera6_007.mp4
